In [ ]:
import math
from scipy.optimize import bisect
from scipy.special import gammaincinv
from scipy.stats import norm

In [ ]:
# Example 1: Consider a bulk carrier located 1.6 km from an H2a hydrophone. The bulk carrier is underway, surrounded by moderate shipping activity, and enjoying fair weather with a steady wind of 5 m/s. What is the SNR at the hydrophone due to the sound signal produced by the bulk carrier?

In [ ]:
# From the problem statemnt, note the distance between the source and receiver
distance = 1600  # m

# From Fig. 4. of McKenna’s article, assume the bulk carrier produces a source density of 160 dB re μPa^2m^2/Hz from 60 to 200 Hz
source_density = 160  # dB re μPa^2m^2/Hz
source_bandwidth = 140  # Hz

# From Fig. 3.19 of Abraham’s text in the frequency band from 60 to 200 Hz, assume shipping noise dominates other noise sources, giving a noise level of 75 dB re μPa^2/Hz
noise_density = 75  # dB re μPa^2/Hz
noise_bandwith = source_bandwidth  # Hz

In [ ]:
# Compute the source level
SL = source_density + 10 * math.log10(source_bandwidth)  # dB re μPa^2m^2

In [ ]:
SL

In [ ]:
# Compute the propation loss
# Note: In deep water the propagation loss can be represented by spherical spreading
PL = 20 * math.log10(distance)  # dB re m^2

In [ ]:
PL

In [ ]:
# Compute the noise level
NL = noise_density + 10 * math.log10(noise_bandwith)  # dB re μPa^2

In [ ]:
NL

In [ ]:
# Since only a single hydrophone is used, note the array gain
AG = 0

In [ ]:
# Use the passive sonar equation to obtain the SNR at the hydrophone
SNR_h = SL - PL - NL + AG

In [ ]:
SNR_h

In [ ]:
# Example 2: Interpreting SNR_h in Example 1 as the detection threshold of the system, what is the operating point of the system if the required probability of false alarm P_f = 0.10?

In [ ]:
# From the problem statement, note probability of false alarm, and detection threshold
P_f = 0.10
SNR_bar_d = SNR_h

In [ ]:
# Rearrange (9.36) to obtain the probability of detection
P_d = math.exp(math.log(P_f)/(1+10**(SNR_bar_d / 10)))

In [ ]:
P_d

In [ ]:
# Example 3: Suppose the bulk carrier of Example 1 moves to a distance of 16 km from the H2a hydrophone. Interpreting the resulting SNR at the hydrophone as the detection threshold of the system, how many independent and identically distributed samples need to be incoherently integrated to obtain the operating point of Example Two?

In [ ]:
# From the problem statement, note the distance between the source and receiver
distance = 16000  # m

In [ ]:
# Compute the propagation loss
# Note: In deep water the propagation loss can be represented by spherical spreading
PL = 20 * math.log10(distance)  # dB re m^2

In [ ]:
PL

In [ ]:
# Use the passive sonar equation to obtain the SNR at the hydrophone, and interpret the result as the detection threshold of the system
SNR_bar_d = SL - PL - NL + AG

In [ ]:
SNR_bar_d

In [ ]:
# Use the notation before (9.40), and then rearrange (9.44) to obtain the approximate number of samples
phi_f = norm.ppf(1 - P_f)
phi_d = norm.ppf(1 - P_d)
M_0 = int(10**((10 * math.log10(phi_f - phi_d) - SNR_bar_d) / 5))

In [ ]:
M_0

In [ ]:
# Define the exact soltuion (2.81)
def snr_bar_d(M, P_f, P_d):
    return 10 * math.log10((gammaincinv(int(M), 1 - P_f) / gammaincinv(int(M), 1 - P_d)) - 1)

In [ ]:
# Use bisection to find the root of the exact solution
def f(M, P_f, P_d, SNR_bar_d):
    return snr_bar_d(M, P_f, P_d) - SNR_bar_d
M_1 = int(bisect(f, M_0, M_0 * 10, args=(P_f, P_d, SNR_bar_d)))

In [ ]:
M_1

In [ ]:
# Check the detection threshold
SNR_bar_d = snr_bar_d(M_1, P_f, P_d)

In [ ]:
SNR_bar_d